In [62]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [4]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.7 MB/s eta 0:00:00


In [5]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.prompts import load_prompt
# from pymongo import MongoClient
# from pymongo.mongo_client import MongoClient
# from pymongo.server_api import ServerApi
import json
import time
import datetime
import pypdf

In [6]:
prompt_template = "templates/template.json"
faiss_index = "faiss_index"
data_source = "data/about_bhavna.csv"
pdf_source = "data/Bhavna_Matwani_Resume.pdf"

In [7]:
def is_valid_json(data):
    try:
        json.loads(data)
        return True
    except json.JSONDecodeError:
        return False

In [8]:
import os
os.environ['OPENAI_API_KEY'] = "Enter key here"

In [10]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00


In [11]:
embeddings = OpenAIEmbeddings()

In [12]:
pdf_loader = PyPDFLoader(pdf_source)
pdf_data = pdf_loader.load()

In [13]:
pdf_data

[Document(page_content='Bhavna Matwani\nbhavna.matwani@nyu.edu ●\uf0e1bhavna-matwani ●\uf09bbhavna-matwani ●+1 (551) 344-7059\nEDUCATION New York University, Courant Institute of Mathematical Sciences , New York City, New York\nMaster of Science in Computer Science | GPA: 3.867/4.0 Sep 2022 – May 2024\nSardar Vallabhbhai National Institute of Technology (SVNIT) , Surat, India\nBachelor of Technology in Electrical Engineering | GPA: 8.86/10.0 Jul 2018 – May 2022\nTECHNICAL\nSKILLSLanguages - C++, Python, TypeScript, Javascript, C, MATLAB, SQL, HTML, Java, Scala, Ocaml, Embedded C\nFrameworks/ Libraries - Pytorch, Keras, TensorFlow, OpenCV , Pandas, OpenAI\nTools - Node.js, React, Angular, Spring Boot, Maven, GitHub, Docker, Kubernetes, OpenShift, Swagger/OpenAPI, Flask, GitHub Actions\nCloud and Big Data Technologies - Google Cloud Platform, AWS, HDFS, MapReduce, Hive, Trino, Zookeeper, Kafka, Selenium\nWORK\nEXPERIENCEPlatform Developer Intern, Vimbly Group May 2023 – Aug 2023\n■Develo

In [14]:
csv_loader = CSVLoader(file_path="data/about_bhavna.csv", encoding="utf-8")
csv_data = csv_loader.load()

In [15]:
csv_data

[Document(page_content="question: What is Bhavna's educational background?\nanswer: Bhavna Matwani completed her Bachelor of Technology in Electrical Engineering from Sardar Vallabhbhai National Institute of Technology, Surat, India in May 2022 with a GPA of 8.86/10.0. She is currently pursuing a Master of Science in Computer Science at New York University, Courant Institute of Mathematical Sciences, with an expected graduation in May 2024 and a current GPA of 3.867/4.0.", metadata={'source': 'about_bhavna.csv', 'row': 0}),
 Document(page_content='question: What technical skills does Bhavna have?\nanswer: Bhavna is skilled in several programming languages including C++, Python, TypeScript, JavaScript, C, MATLAB, SQL, HTML, Java, Scala, and Embedded C. She is also proficient with frameworks and libraries such as Pytorch, Keras, TensorFlow, OpenCV, and Pandas, and tools like Node.js, React, Angular, Spring Boot, Maven, Docker, Kubernetes, OpenShift, and Flask. Additionally, she has exper

In [17]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.8 MB/s eta 0:00:00


In [19]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 36.7 MB/s eta 0:00:00


In [20]:
# Combine PDF and CSV data
data = pdf_data + csv_data

# Create embeddings for the docs
vectors = FAISS.from_documents(data, embeddings)
vectors.save_local("faiss_index")

In [21]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [22]:
prompt = load_prompt("template.json")

In [23]:
retriever = vectors.as_retriever(search_type="similarity", search_kwargs={"k":6, "include_metadata":True, "score_threshold":0.6})
chain = ConversationalRetrievalChain.from_llm(llm=ChatOpenAI(temperature=0.5,model_name='gpt-3.5-turbo', openai_api_key=openai_api_key),
                                                retriever=retriever,return_source_documents=True,verbose=True,chain_type="stuff",
                                                max_tokens_limit=4097, combine_docs_chain_kwargs={"prompt": prompt})


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [24]:
chat_history = []
result = chain({"system":
        "You are a Resume Bot, a comprehensive, interactive resource for exploring Bhavna's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.",
                        "question": "Has Bhavna worked on GenAI?",
                        "chat_history":chat_history})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a Resume Bot, a comprehensive, interactive resource for exploring Bhavna's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge. 
 Human: Take a deep breath and do the following step by step these 4 steps:  
 1. Read the context below 
 2. Answer the question with detail using the provided Help Centre information 
 3. Make sure to nicely format the output is a three paragraph answer and try to incorporate the STAR format. 
 4. Provide 3 examples of questions user can ask about Bhavna   based on the questions from context. Context : 
 ~~~ question: Could you tell me about Bhavna's skills?
answer: Bhavna's skills span across programming, system design, AI and machine learning, with practical applications in software development, cloud technologies, and data analytics. He

In [25]:
result

{'system': "You are a Resume Bot, a comprehensive, interactive resource for exploring Bhavna's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.",
 'question': 'Has Bhavna worked on GenAI?',
 'chat_history': [],
 'answer': '{\n    "answered": true,\n    "response": "Bhavna has not worked specifically on GenAI. Her expertise lies in programming, system design, AI, and machine learning, with a focus on projects like the E-Commerce Shopcarts Microservice and Multi-Runner Segmentation for marathon images. She is currently completing her Master of Science in Computer Science at NYU, where she applies AI and machine learning to solve practical problems.",\n    "questions": [\n        "What projects has Bhavna worked on related to AI and machine learning?",\n        "Can Bhavna provide examples of her programming skills in action?",\n        "How does Bhavna apply her technical expertise to 

In [26]:
result['answer']

'{\n    "answered": true,\n    "response": "Bhavna has not worked specifically on GenAI. Her expertise lies in programming, system design, AI, and machine learning, with a focus on projects like the E-Commerce Shopcarts Microservice and Multi-Runner Segmentation for marathon images. She is currently completing her Master of Science in Computer Science at NYU, where she applies AI and machine learning to solve practical problems.",\n    "questions": [\n        "What projects has Bhavna worked on related to AI and machine learning?",\n        "Can Bhavna provide examples of her programming skills in action?",\n        "How does Bhavna apply her technical expertise to solve real-world problems?"\n    ]\n}'

In [27]:
data = json.loads(result["answer"])

In [28]:
data.get("response")

'Bhavna has not worked specifically on GenAI. Her expertise lies in programming, system design, AI, and machine learning, with a focus on projects like the E-Commerce Shopcarts Microservice and Multi-Runner Segmentation for marathon images. She is currently completing her Master of Science in Computer Science at NYU, where she applies AI and machine learning to solve practical problems.'

In [29]:
data.get("questions")

['What projects has Bhavna worked on related to AI and machine learning?',
 'Can Bhavna provide examples of her programming skills in action?',
 'How does Bhavna apply her technical expertise to solve real-world problems?']

In [30]:
chat_history.append((result["question"], result["answer"]))

In [31]:
chat_history

[('Has Bhavna worked on GenAI?',
  '{\n    "answered": true,\n    "response": "Bhavna has not worked specifically on GenAI. Her expertise lies in programming, system design, AI, and machine learning, with a focus on projects like the E-Commerce Shopcarts Microservice and Multi-Runner Segmentation for marathon images. She is currently completing her Master of Science in Computer Science at NYU, where she applies AI and machine learning to solve practical problems.",\n    "questions": [\n        "What projects has Bhavna worked on related to AI and machine learning?",\n        "Can Bhavna provide examples of her programming skills in action?",\n        "How does Bhavna apply her technical expertise to solve real-world problems?"\n    ]\n}')]

In [32]:
result = chain({"system":
        "You are a Resume Bot, a comprehensive, interactive resource for exploring Harikrishna (Harry) Dev's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.",
                        "question": "What are some of Bhavna's projects",
                        "chat_history":chat_history})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Has Bhavna worked on GenAI?
Assistant: {
    "answered": true,
    "response": "Bhavna has not worked specifically on GenAI. Her expertise lies in programming, system design, AI, and machine learning, with a focus on projects like the E-Commerce Shopcarts Microservice and Multi-Runner Segmentation for marathon images. She is currently completing her Master of Science in Computer Science at NYU, where she applies AI and machine learning to solve practical problems.",
    "questions": [
        "What projects has Bhavna worked on related to AI and machine learning?",
        "Can Bhavna provide examples of her programming skills in action?",
        "How does Bhavna apply her technical expertise to solve real-world problems?"
    ]
}
Follow Up Input: What are som

In [33]:
result["answer"]

'{\n    "answered": true,\n    "response": "Bhavna has worked on several projects related to AI and machine learning, including FrameForesight for predicting video frames using machine learning, Multi-Runner Segmentation for marathon image analysis using YOLOv4 and MASK R-CNN, and Tunes for Expression for facial analysis and song recommendations. These projects showcase Bhavna\'s expertise in applying AI and machine learning to solve practical problems with innovative solutions.",\n    "questions": [\n        "What are Bhavna\'s key achievements in AI and machine learning projects?",\n        "How has Bhavna\'s experience in AI and machine learning projects contributed to her skill development?",\n        "Can you provide more details on the impact of Bhavna\'s AI and machine learning projects?"\n    ]\n}'